# TxtAI got skills

This example will demonstrate how to use `txtai` agents with [`skill.md`](https://agentskills.io/specification) files.

We'll setup a `skill.md` file with details on how to use TxtAI and run a series of agent requests.

Let's get started!

# Install dependencies

Install `txtai` and all dependencies.

In [ ]:
%%capture
!pip install git+https://github.com/neuml/txtai#egg=txtai[agent]

# Define a `skill.md` file

Next, we'll create our `skill.md` file. This file has examples on how to build embeddings databases, how to use re-ranker pipelines, RAG pipelines and more.

The upside of a `skill.md` file vs an `agents.md` file is that it can be dynamically added to the agent context. The description helps the agent decide if the `skill` is necessary given the request. Think of it like a dynamic knowledge base that's easy to modify.

In [ ]:
%%writefile skill.md
---
name: txtai
description: Examples on how to build txtai embeddings databases, txtai RAG pipelines, txtai reranker pipelines and txtai translation pipelines
---

# Build an embeddings database

```python
from txtai import Embeddings

# Create embeddings model, backed by sentence-transformers & transformers
embeddings = Embeddings(path="sentence-transformers/nli-mpnet-base-v2")

data = [
  "US tops 5 million confirmed virus cases",
  "Canada's last fully intact ice shelf has suddenly collapsed, " +
  "forming a Manhattan-sized iceberg",
  "Beijing mobilises invasion craft along coast as Taiwan tensions escalate",
  "The National Park Service warns against sacrificing slower friends " +
  "in a bear attack",
  "Maine man wins $1M from $25 lottery ticket",
  "Make huge profits without work, earn up to $100,000 a day"
]

# Index the list of text
embeddings.index(data)
```

# Search an embeddings database

```python
embeddings.search("Search query")
```

# Build a RAG pipeline

```python
from txtai import Embeddings, RAG

# Input data
data = [
  "US tops 5 million confirmed virus cases",
  "Canada's last fully intact ice shelf has suddenly collapsed, " +
  "forming a Manhattan-sized iceberg",
  "Beijing mobilises invasion craft along coast as Taiwan tensions escalate",
  "The National Park Service warns against sacrificing slower friends " +
  "in a bear attack",
  "Maine man wins $1M from $25 lottery ticket",
  "Make huge profits without work, earn up to $100,000 a day"
]

# Build embeddings index
embeddings = Embeddings(content=True)
embeddings.index(data)

# Create the RAG pipeline
rag = RAG(embeddings, "Qwen/Qwen3-0.6B", template="""
  Answer the following question using the provided context.

  Question:
  {question}

  Context:
  {context}
""")

# Run RAG pipeline
rag("What was won?")
```

# Translate text from English into French

```python
from txtai.pipeline import Translation

# Create and run pipeline
translate = Translation()
translate("This is a test translation", "fr")
```

# Re-ranker pipeline

```python
from txtai import Embeddings
from txtai.pipeline import Reranker, Similarity

# Embeddings instance
embeddings = Embeddings()
embeddings.load(provider="huggingface-hub", container="neuml/txtai-wikipedia")

# Similarity instance
similarity = Similarity(path="colbert-ir/colbertv2.0", lateencode=True)

# Reranking pipeline
reranker = Reranker(embeddings, similarity)
reranker("Tell me about AI")
```

# Query for TxtAI questions

Now, let's try this out and see if the LLM is smart enough to use the defined skill vs. going out on the web.

Let's setup the scaffolding code to create and run an agent. We'll use a [Qwen3 4B non-thinking LLM](https://huggingface.co/Qwen/Qwen3-4B-Instruct-2507) as the agent's model. We'll add the `websearch` and `webview` tools to the agent along with the `skill.md` file previously created.

Additionally, we'll add a sliding window of the last 2 responses as "agent memory". This will help create a rolling dialogue. 

In [ ]:
from txtai import Agent
from IPython.display import display, Markdown

def run(query, reset=False):
    answer = agent(query, maxlength=50000, reset=reset)
    display(Markdown(answer))

agent = Agent(
    model="Qwen/Qwen3-4B-Instruct-2507",
    tools=["websearch", "webview", "skill.md"],
    memory=2,
    verbosity_level=0
)

First, we'll ask how to build a TxtAI embeddings database.

In [2]:
run("Write a txtai embeddings program that indexes data")

To create a txtai embeddings program that indexes data, use the following Python code:

```python
from txtai import Embeddings

# Create embeddings model using sentence-transformers
embeddings = Embeddings(path="sentence-transformers/nli-mpnet-base-v2")

# Sample data to index
data = [
  "US tops 5 million confirmed virus cases",
  "Canada's last fully intact ice shelf has suddenly collapsed, " +
  "forming a Manhattan-sized iceberg",
  "Beijing mobilises invasion craft along coast as Taiwan tensions escalate",
  "The National Park Service warns against sacrificing slower friends " +
  "in a bear attack",
  "Maine man wins $1M from $25 lottery ticket",
  "Make huge profits without work, earn up to $100,000 a day"
]

# Index the data
embeddings.index(data)
```

This program initializes an embeddings database using the `sentence-transformers/nli-mpnet-base-v2` model and indexes a list of text data. The indexed data can later be searched or used in other applications like retrieval, RAG, or translation.

The Agent pulled the correct section from the `skill.md` file. Now, let's look for an example on how to use a re-ranker pipeline.

In [3]:
run("Write a txtai re-ranker pipeline")

```python
from txtai import Embeddings
from txtai.pipeline import Reranker, Similarity

# Embeddings instance
embeddings = Embeddings()
embeddings.load(provider="huggingface-hub", container="neuml/txtai-wikipedia")

# Similarity instance
similarity = Similarity(path="colbert-ir/colbertv2.0", lateencode=True)

# Reranking pipeline
reranker = Reranker(embeddings, similarity)
reranker("Tell me about AI")
```

Great! This also works. But remember we have access to an LLM here. It doesn't have to blindly just pull the text. Let's ask it to modify the last example.

In [4]:
run("Update the similarity path to use another reranker model. Disable lateencode.")

```python
from txtai import Embeddings
from txtai.pipeline import Reranker, Similarity

# Embeddings instance
embeddings = Embeddings()
embeddings.load(provider="huggingface-hub", container="neuml/txtai-wikipedia")

# Similarity instance with a different reranker model and lateencode disabled
similarity = Similarity(path="BAAI/bge-reranker-base", lateencode=False)

# Reranking pipeline
reranker = Reranker(embeddings, similarity)
reranker("Tell me about AI")
```

Notice it just edited the code with a different reranker model and even added a comment to note this change.

We can also clear the rolling dialogue and start fresh. 

In [5]:
run("Write a txtai program that translate text from English to Spanish", reset=True)

```python
from txtai.pipeline import Translation

# Create and run pipeline for English to Spanish translation
translate = Translation()
translated_text = translate("This is a test translation", "es")
print(translated_text)
```

# Wrapping up

This example shows how to add a `skill.md` file to `txtai` agents. Go give it a try!